Plot the correlation coefficient matrix and the cloud of points for every pair of variables. This helps to familiarize myself with the data, it allows me to check how strongly correlated some variables are and what type of correlation is there.

In [73]:
import numpy as np
import matplotlib.pyplot as plt

files = ['data-hl.txt','data-hs.txt','data-ne.txt','data-sl.txt','data-ss.txt']
data = np.zeros([5*300,42])

print 'read data files...'
i = 0
for file in files:
    j = 0
    for line in open(file,'r'):
        if line.split('\t')[0] != 'sclass':
            values = np.array(line.split('\t'))
            values[values == 'NA'] = 'nan'
            values[values == 'NA\n'] = 'nan'
            # save the class: points in data-hl.txt have class 0, points in data-hs.txt are in class 1, etc.
            data[i*300+j,0] = i
            data[i*300+j,1:] = values[1:].astype(float)
            j = j + 1
        else:
            header = line.split('\t')[1:]
    i = i + 1 
print 'done'    

print 'prepare the plots...'

# correlation coefficient matrix, masking out nans (didn't find a way to do this without for loops)
corr_coef = np.zeros([41,41])
for i in range(41):
    for j in range(41):
        a = data[:,i+1]
        b = data[:,j+1]
        mask_a = np.isfinite(a)
        mask_b = np.isfinite(b)
        mask = mask_a & mask_b
        corr_coef[i,j] = np.corrcoef([a[mask],b[mask]])[0,1]

plt.title('white - no corr., red - positive corr., blue - negative corr.')
plt.imshow(corr_coef,cmap="bwr",interpolation="nearest")
plt.savefig('corr_coef.png')
plt.close()

# covariance matrix, masking out nans (didn't find a way to do this without for loops)
cov = np.zeros([41,41])
for i in range(41):
    for j in range(41):
        a = data[:,i+1]
        b = data[:,j+1]
        mask_a = np.isfinite(a)
        mask_b = np.isfinite(b)
        mask = mask_a & mask_b
        cov[i,j] = np.cov([a[mask],b[mask]])[0,1]

plt.imshow(cov,cmap="bwr",interpolation="nearest")
plt.savefig('covariance.png')
plt.close()

for i in range(41):
    for j in range(i):
        # i+1 and j+1 to skip the first row of classes
        for k in range(5):
            mask = (data[:,0] == k)
            plt.plot(data[mask,i+1],data[mask,j+1],'+')
        plt.xlabel(header[i])
        plt.ylabel(header[j])
        plt.savefig('imgs/'+header[i]+'-'+header[j]+'.png')
        plt.close()
        
print 'done'




read data files...
done
prepare the plots...
done


Class separability or class discriminatory power of the features.
Calculate the Fisher's discriminant ratio for each feature and rank the features in descending order. The first feature in the list has the highest class separability of all features. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

standardize = True

files = ['data-hl.txt','data-hs.txt','data-ne.txt','data-sl.txt','data-ss.txt']
data = np.zeros([5*300,42])

print 'read data files...'
i = 0
for file in files:
    j = 0
    for line in open(file,'r'):
        if line.split('\t')[0] != 'sclass':
            values = np.array(line.split('\t'))
            values[values == 'NA'] = 'nan'
            values[values == 'NA\n'] = 'nan'
            # save the class: points in data-hl.txt have class 0, points in data-hs.txt are in class 1, etc.
            data[i*300+j,0] = i
            data[i*300+j,1:] = values[1:].astype(float)
            j = j + 1
        else:
            header = line.split('\t')[1:]
            header = np.array(header)
    i = i + 1 
print 'done'    

if standardize:
    print 'standardize data...'
    # This is necessary because different features have different dynamic ranges. Standardization brings all features 
    # to the same scale.

    mean = np.mean(data,axis=0)
    std = np.std(data,axis=0)

    data_standard = np.zeros(np.shape(data))
    data_mask = np.zeros(np.shape(data))
    # save the classes
    data_standard[:,0] = data[:,0]
    data_mask = np.isfinite(data)
    # standardize data
    for i in range(41):
        a = data[:,i+1]
        mask_a = data_mask[:,i+1]
        mean = np.mean(a[mask_a]) 
        std = np.std(a[mask_a])

        data_standard[mask_a,i+1] = (a[mask_a] - mean)/std

    print 'done'

    print 'calculate Fisher discriminant ratio (FDR)...'
    # FDR = sum sum (mu_i - mu_j)^2 / (sigma_i^2 + sigma_j^2), where i and j are different classes, mu is the mean, 
    # sigma is the variance. Sum goes over all i-j pairs excluding i=j. 
    # The idea is that features with large differences in the class-specific means and small variances in each class
    # are better at distinguishing classes.
    # For more details on FDR see e.g., Lin et al., J. Chem. Inf. Comput. Sci. 2004, 44, 76-87

    FDR = np.zeros(41)
    for i in range(41):
        FDR_sum = 0e0
        for j in range(5):
            mask_j = data_mask[:,i+1] & (data_standard[:,0] == j)
            for k in range(j):
                mask_k = data_mask[:,i+1] & (data_standard[:,0] == k)

                mu_j = np.mean(data_standard[mask_j,i+1])
                mu_k = np.mean(data_standard[mask_k,i+1])
                sigma_j = np.var(data_standard[mask_j,i+1])
                sigma_k = np.var(data_standard[mask_k,i+1])

                FDR_sum = FDR_sum + (mu_j-mu_k)**2e0 / (sigma_j**2e0 + sigma_k**2e0)
        # check the values     
        #print i,FDR_sum
        #for j in range(5):
        #    mask_j = data_mask[:,i+1] & (data_standard[:,0] == j)
        #    print '   ',np.mean(data_standard[mask_j,i+1])/np.var(data_standard[mask_j,i+1])
        FDR[i] = FDR_sum
else:
    print 'calculate Fisher discriminant ratio (FDR)...'
    # FDR = sum sum (mu_i - mu_j)^2 / (sigma_i^2 + sigma_j^2), where i and j are different classes, mu is the mean, 
    # sigma is the variance. Sum goes over all i-j pairs excluding i=j. 
    # The idea is that features with large differences in the class-specific means and small variances in each class
    # are better at distinguishing classes.
    # For more details on FDR see e.g., Lin et al., J. Chem. Inf. Comput. Sci. 2004, 44, 76-87
    
    data_mask = np.zeros(np.shape(data))
    data_mask = np.isfinite(data)
    
    FDR = np.zeros(41)
    for i in range(41):
        FDR_sum = 0e0
        for j in range(5):
            mask_j = data_mask[:,i+1] & (data[:,0] == j)
            for k in range(j):
                mask_k = data_mask[:,i+1] & (data[:,0] == k)

                mu_j = np.mean(data[mask_j,i+1])
                mu_k = np.mean(data[mask_k,i+1])
                sigma_j = np.var(data[mask_j,i+1])
                sigma_k = np.var(data[mask_k,i+1])

                FDR_sum = FDR_sum + (mu_j-mu_k)**2e0 / (sigma_j**2e0 + sigma_k**2e0)
        # check the values     
        #print i,FDR_sum
        #for j in range(5):
        #    mask_j = data_mask[:,i+1] & (data_standard[:,0] == j)
        #    print '   ',np.mean(data_standard[mask_j,i+1])/np.var(data_standard[mask_j,i+1])
        FDR[i] = FDR_sum
    
    
print 'done'

print 'rank the features...'
indx_sorted = np.argsort(FDR)[::-1]
print '   features sorted in order of how well they discriminate between different classes (first item is best):'
print '  ',header[indx_sorted]

# add 1 and insert 0 to the first place to keep the class in.
indx_sorted = np.insert(indx_sorted+1,0,0)

print 'done'


read data files...
done
standardize data...
done
calculate Fisher discriminant ratio (FDR)...
done
rank the features...
   features sorted in order of how well they discriminate between different classes (first item is best):
   ['ThetaPi_1' 'H2.H1_1' 'H1_1' 'Theta1Pi_1' 'H12_1' 'ThetaS_1' 'DAF_1'
 'Theta1S_1' 'TajD_1' 'FuLiF_1' 'Theta1L_1' 'FuLiD_1' 'DXPEHH_12'
 'FuLiF1_1' 'TajD1_1' 'DXPEHH_13\n' 'FuLiD1_1' 'FayWuH_1' 'FST_1'
 'XPEHH_12' 'SL1_1' 'H2_1' 'XPEHH_13' 'ThetaL_1' 'SL0_1' 'iHH0_1'
 'Theta1H_1' 'DDAF_1' 'Theta1Xi_1' 'nSL_1' 'iHS_1' 'FayWuH1_1' 'DnSL_1'
 'iHH1_1' 'ZengE1_1' 'MAF_1' 'ZengE_1' 'ZA_1' 'ThetaXi_1' 'DiHH_1'
 'ThetaH_1']
done


use an SVM to do classification
separate the data into training, cross validation, and test data sets (60-20-20%).
make a loop through successively less features:
    - use all features to find the best values for C and gamma in the SVM
    - fix the best C and gamma values and successively remove a feature that is least discriminative
    - check what number of features give the best score in classification

In [70]:
# run the previous cell first!
from sklearn.svm import SVC

# do everything 100 times to characterize the noise

print 'Train the SVM and find the best C and gamma parameters...'
C_collect = []
gamma_collect = []
for i in range(100):
    print '   round',i

    # I can only use those data points that do not contain NAs
    mask_to_use = np.all(np.isfinite(data),axis=1)
    
    if standardize:
        data_to_use = data_standard[mask_to_use,:]
    else:
        data_to_use = data[mask_to_use,:]
    
    # nr of data points left
    n = np.shape(data_to_use)[0]
    
    # shuffle and divide up the data
    indx = np.arange(n)
    np.random.shuffle(indx)
    
    X_train = data_to_use[indx[:n*0.6],1:]
    Y_train = data_to_use[indx[:n*0.6],0]
    
    X_test = data_to_use[indx[n*0.6:n*0.8],1:]
    Y_test = data_to_use[indx[n*0.6:n*0.8],0]
    
    X_CV = data_to_use[indx[n*0.8:],1:]
    Y_CV = data_to_use[indx[n*0.8:],0]
    
    # parameter ranges for the SVM
    C = 10e0**(np.linspace(-8e0,3e0,12))
    gamma = 10e0**(np.linspace(-5e0,6e0,12))
    
    # arrays to store scores and best parameters
    train_score = np.zeros([len(C),len(gamma)])
    test_score = np.zeros([len(C),len(gamma)])
    C_array = np.zeros([len(C),len(gamma)])
    gamma_array = np.zeros([len(C),len(gamma)])
    # loop through the C and gamma arrays
    for i in range(len(C)):
        for j in range(len(gamma)):
            SVM = SVC(kernel='rbf', C=C[i], gamma=gamma[j]).fit(X_train, Y_train)
            train_score[i,j] = SVM.score(X_train,Y_train)
            test_score[i,j] = SVM.score(X_test,Y_test)
            C_array[i,j] = C[i]
            gamma_array[i,j] = gamma[j]

    # find the C and gamma parameters that give max score.
    # if there are multiple parameter configuration giving max score, the first one of these is used below 
    best_params = np.where(test_score == np.max(test_score))
    print '      best C value(s):',C[best_params[0]]
    print '      best gamma value(s):',gamma[best_params[1]]
    print '      max test score:',np.max(test_score)
    # calculate the cross validation score
    for j in range(len(best_params[0])):
        print '      the cross validation score for C='+str(C[best_params[0][j]])+' and gamma='+str(gamma[best_params[1][j]])+':'
        SVM = SVC(kernel='rbf', C=C[best_params[0][j]], gamma=gamma[best_params[1][j]]).fit(X_train, Y_train)
        CV_score = SVM.score(X_CV,Y_CV)
        print '         ',CV_score
    
    C_collect.append(C[best_params[0]])
    gamma_collect.append(gamma[best_params[1]])

print 'done'

# flatten the *_collect lists and find the most frequently occuring elements
C_flat = [item for sublist in C_collect for item in sublist]
gamma_flat = [item for sublist in gamma_collect for item in sublist]

combined = [t for t in zip(C_flat,gamma_flat)]
(C, gamma) = max(set(combined), key=combined.count)

print 'The best C and gamma values are',C,'and',gamma,', respectively.'

print 'Remove one feature at a time (the least discriminative one) and calculate the classification score...'

n_sim = 10
train_score = np.zeros([n_sim,40])
test_score = np.zeros([n_sim,40])

for i in range(n_sim):
    if i%10 == 0:
        print '   ',i
    for j in range(40):        
        # I can only use those data points that do not contain NAs
        mask_to_use = np.all(np.isfinite(data[:,indx_sorted[:-(j+1)]]),axis=1)
        
        if standardize:
            data_to_use = data_standard[:,indx_sorted[:-(j+1)]]  
        else:
            data_to_use = data[:,indx_sorted[:-(j+1)]]
        
        data_to_use = data_to_use[mask_to_use,:]
                        
        # nr of data points left
        n = np.shape(data_to_use)[0]
                
        # shuffle and divide up the data
        indx = np.arange(n)
        np.random.shuffle(indx)

        X_train = data_to_use[indx[:n*0.75],1:]
        Y_train = data_to_use[indx[:n*0.75],0]

        X_test = data_to_use[indx[n*0.75:],1:]
        Y_test = data_to_use[indx[n*0.75:],0]

        SVM = SVC(kernel='rbf', C=C, gamma=gamma).fit(X_train, Y_train)
        train_score[i,j] = SVM.score(X_train,Y_train)
        test_score[i,j] = SVM.score(X_test,Y_test)
                

# make a plot            
plt.close()
plt.ylim([0,80])
plt.errorbar(41 - (np.arange(40)+1),np.average(test_score,axis=0)*100e0,yerr=np.std(test_score,axis=0)*100,fmt='o')
plt.xlabel('nr. of features used')
plt.ylabel('test score [%]')
if standardize:
    plt.savefig('classification_standardized_'+str(n_sim)+'.png')
else:
    plt.savefig('classification_'+str(n_sim)+'.png')
plt.close()
print 'done'


Remove one feature at a time (the least discriminative one) and calculate the classification score...
    0
done


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:29: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:32: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Using gaussian naive bayes classifiers

In [2]:
# trying a naive bayes classifier of sklearn
# it does not handle missing data!

from sklearn.naive_bayes import GaussianNB

if standardize:
    data_to_use = data_standard
else:
    data_to_use = data

data_mask = np.zeros(np.shape(data))
data_mask = np.isfinite(data)
    
# shuffle and divide up the data
n = np.shape(data_to_use)[0]
indx = np.arange(n)
np.random.shuffle(indx)

X_train = data_to_use[indx[:n*0.6],1:]
Y_train = data_to_use[indx[:n*0.6],0]

X_test = data_to_use[indx[n*0.6:n*0.8],1:]
Y_test = data_to_use[indx[n*0.6:n*0.8],0]

X_CV = data_to_use[indx[n*0.8:],1:]
Y_CV = data_to_use[indx[n*0.8:],0]

gnb = GaussianNB()

y_pred = gnb.fit(X_train[np.isfinite(X_train)], Y_train).predict(X_test[np.isfinite(X_test)])

print (Y_test != y_pred).sum()


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:20: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:22: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:25: De

ValueError: Found arrays with inconsistent numbers of samples: [  1 900]

In [96]:
# gaussian naive bayes classifier from scratch that handles missing data
# not optimized!
# run the FDR ranking cell first!

n_sim = 300

test_score = np.zeros([n_sim,40])

# run n_sim different simulations to get a feeling of random effects
for ii in range(n_sim):

    if ii%10 == 0:
        print ii
    
    # remove one feature at a time (the least disciminative one)
    for jj in range(40):

        if standardize:
            data_to_use = data_standard[:,indx_sorted[:-(jj+1)]]
        else:
            data_to_use = data[:,indx_sorted[:-(jj+1)]] 

        
        # arrays used to summarize data
        # size of arrays: [nr of classes, nr of features]
        nr_classes = len(np.unique(data_to_use[:,0]))
        nr_features = np.shape(data_to_use[:,1:])[1]

        mean = np.zeros([nr_classes,nr_features])
        std = np.zeros(np.shape(mean))
        # correction factor for the probabilities: its value is the fraction of data points used to calculate the mean and std
        #   1 if all points are used, 0.1 if only 10% of points are used
        corr_factor = np.zeros(np.shape(mean))


        # shuffle and divide up the data
        n = np.shape(data_to_use)[0]
        indx = np.arange(n)
        np.random.shuffle(indx)

        X_train = data_to_use[indx[:n*0.75],1:]
        Y_train = data_to_use[indx[:n*0.75],0]

        X_test = data_to_use[indx[n*0.75:],1:]
        Y_test = data_to_use[indx[n*0.75:],0]


        # fill up the mean, std, and corr_factor arrays
        for i in range(nr_classes):
            mask = Y_train == i
            points_in_class = X_train[mask,:]

            for j in range(nr_features):
                feature_j = points_in_class[:,j]
                mask = np.isfinite(feature_j)

                mean[i,j] = np.mean(feature_j[mask])
                std[i,j] = np.std(feature_j[mask])
                corr_factor[i,j] = float(np.sum(mask)) / float(len(feature_j))


        # loop through the test points and estimate the most likely class
        score = 0e0
        for i in range(len(Y_test)): 
            
            class_prob = np.zeros(nr_classes)
            for j in range(nr_classes):
                
                # calculate the log probabilities that this data point was sampled from class j
                # adding log probabilities is better than multiplying probabilities 
                log_factor = np.log(corr_factor[j,:] / (np.sqrt(2e0*np.pi)*std[j,:]))
                log_exponent = -((X_test[i,:]-mean[j,:])**2e0/(2e0*std[j,:]**2e0))
                log_prob = log_factor + log_exponent
                class_prob[j] = np.sum(log_prob[np.isfinite(log_prob)])

            if np.argmax(class_prob) == Y_test[i]:
                score = score + 1

        test_score[ii,jj] = score / len(Y_test)


# make a plot            
plt.close()
plt.ylim([0,80])
plt.errorbar(41 - (np.arange(40)+1),np.average(test_score,axis=0)*100e0,yerr=np.std(test_score,axis=0)*100,fmt='o')
plt.xlabel('nr. of features used')
plt.ylabel('test score [%]')
if standardize:
    plt.savefig('naive_bayes_standardized_'+str(n_sim)+'.png')
else:
    plt.savefig('naive_bayes_'+str(n_sim)+'.png')
plt.close()
print 'done'


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
done


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:40: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:41: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:43: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:44: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [18]:
# general naive bayes classifier
# the true probability distribution of the features is estimated using a gaussian kernel density estimator
# https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/
# scipy's kde is fastest for a few 100 data points.

from scipy.stats import gaussian_kde

n_sim = 100

test_score = np.zeros([n_sim,40])

# test what bandwidth value gives best scores
bandwidth = 10e0**(np.linspace(-2,-1,num=6,endpoint = True))

for bw in bandwidth:
    print bw
    
    # run n_sim different simulations to get a feeling of random effects
    for ii in range(n_sim):
        if ii%10 == 0:
            print '   ',ii

        # remove one feature at a time (the least disciminative one)
        for jj in range(40):

            if standardize:
                data_to_use = data_standard[:,indx_sorted[:-(jj+1)]]
            else:
                data_to_use = data[:,indx_sorted[:-(jj+1)]] 

            nr_classes = len(np.unique(data_to_use[:,0]))
            nr_features = np.shape(data_to_use[:,1:])[1]

            # shuffle and divide up the data
            n = np.shape(data_to_use)[0]
            indx = np.arange(n)
            np.random.shuffle(indx)

            X_train = data_to_use[indx[:n*0.75],1:]
            Y_train = data_to_use[indx[:n*0.75],0]

            X_test = data_to_use[indx[n*0.75:],1:]
            Y_test = data_to_use[indx[n*0.75:],0]


            # collect kernels for each class and feature
            kernels = []
            for i in range(nr_classes):
                mask = Y_train == i
                points_in_class = X_train[mask,:]

                kernels_class_i = []
                
                for j in range(nr_features):
                    feature_j = points_in_class[:,j]
                    mask = np.isfinite(feature_j)

                    kernels_class_i.append(gaussian_kde(feature_j[mask],bw_method = bw))

                kernels.append(kernels_class_i)


            # loop through the test points and estimate the most likely class
            score = 0e0
            for i in range(len(Y_test)): 

                class_prob = np.zeros(nr_classes)
                for j in range(nr_classes):
                    kernels_class_j = kernels[j]
                    
                    #for k in range(nr_features):
                    #    kernel = kernels_class_j[k]
                    #    class_prob[j] = class_prob[j] + kernel.logpdf(X_test[i,k])
                        
                    class_prob[j] = np.sum([kernel.logpdf(point) for kernel, point in zip(kernels_class_j, X_test[i,:])])

                if np.argmax(class_prob) == Y_test[i]:
                    score = score + 1

            test_score[ii,jj] = score / len(Y_test)

    # make a plot            
    plt.close()
    plt.ylim([0,80])
    plt.errorbar(41 - (np.arange(40)+1),np.average(test_score,axis=0)*100e0,yerr=np.std(test_score,axis=0)*100,fmt='o')
    plt.xlabel('nr. of features used')
    plt.ylabel('test score [%]')
    if standardize:
        plt.savefig('general_naive_bayes_standardized_bw'+str(bw)+'_'+str(n_sim)+'.png')
    else:
        plt.savefig('general_naive_bayes_bw'+str(bw)+'_'+str(n_sim)+'.png')
    plt.close()
    print 'done'



0.01
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done
0.0158489319246
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done
0.0251188643151
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done
0.0398107170553
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done
0.063095734448
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done
0.1
    0
    10
    20
    30
    40
    50
    60
    70
    80
    90
done


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:40: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:42: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:43: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
